# ngram 기반 간단한 텍스트 생성기

In [43]:
import nltk
from nltk.util import ngrams
from collections import Counter
import random
from string import punctuation

In [44]:
def generate_text_bigram(seed, unigram_Freq, bigram_freq, max_len=10, max_sent=5):
    current_word = seed
    generated = [current_word]
    sent_count = 0

    for _ in range(max_len - 1):
        candidates = [(bigram, freq) for bigram, freq in bigram_freq.items() if bigram[0] == current_word]
        if not candidates:
            break
        
        words, freqs = zip(*[(bigram[1], freq) for bigram, freq in candidates])
        total = sum(freqs)
        probs = [f / total for f in freqs]
        
        next_word = random.choices(words, weights=probs)[0]
        if next_word in punctuation:
            generated[-1] += next_word
            if next_word == '.':
                sent_count += 1
                if sent_count == max_sent:
                    return " ".join(generated)
        else:
            generated.append(next_word)
        current_word = next_word
    

    return " ".join(generated)
    

In [ ]:
def generate_text_trigram(seed_bigram, trigram_freq, max_len=50, max_sent=5):
    w1, w2 = seed_bigram
    generated = [w1, w2]
    sent_count = 0

    for _ in range(max_len - 2):
        # (w1, w2)가 앞 두 단어인 trigram 후보 찾기
        candidates = [(w3, freq) for (a, b, w3), freq in trigram_freq.items() if a == w1 and b == w2]
        if not candidates:
            break

        words, freqs = zip(*candidates)
        total = sum(freqs)
        probs = [f / total for f in freqs]

        next_word = random.choices(words, weights=probs)[0]

        # 문장 부호 처리
        if next_word in punctuation:
            generated[-1] += next_word
            if next_word == '.':
                sent_count += 1
                if sent_count == max_sent:
                    break
        else:
            generated.append(next_word)

        # 다음 trigram 예측을 위해 앞 두 단어 갱신
        w1, w2 = w2, next_word

    return " ".join(generated)


In [69]:


train_text = """
인간의 언어는 복잡하고 미묘하다. 같은 단어라도 문맥에 따라 전혀 다른 의미로 해석될 수 있다. 언어 모델을 만드는 과정은 이러한 차이를 수학적으로 포착하는 시도와 같다. 
단어들의 연속에서 규칙을 찾고, 의미를 예측하며, 다음 단어를 결정하는 과정은 단순한 계산이 아니라 사고의 모방이다. 컴퓨터가 문장을 이해하기 위해서는 통계와 확률, 그리고 수많은 예시가 필요하다. 
사람이 언어를 배우듯이 기계도 데이터를 통해 언어의 패턴을 익힌다. 하지만 기계는 감정을 느끼지 못하고 맥락을 완벽히 파악하지 못한다. 
그럼에도 불구하고 언어모델은 점점 사람의 사고를 닮아가고 있다. 문장을 완성하고, 질문에 답하며, 감정이 섞인 표현조차 흉내 낸다. 
이러한 기술의 발전은 단순한 계산 능력의 향상이 아니라, 인간이 언어를 이해하는 방식을 역으로 탐구하는 과정이기도 하다. 
언어는 단순한 소통의 도구가 아니라 인간 사고의 본질이다. 언어를 분석한다는 것은 곧 인간의 생각을 해부하는 일이다. 그리고 인공지능은 지금 그 해부의 최전선에 서 있다. 
현대 사회에서 기술은 인간의 일상을 지배하고 있다. 눈을 뜨자마자 스마트폰을 확인하고, 일을 시작하기 전 이메일을 열며, 하루의 대부분을 디지털 화면 앞에서 보낸다. 
인간의 행동은 점점 더 데이터화되고, 그 데이터는 거대한 알고리즘의 재료가 된다. 사람들은 기술을 사용한다고 생각하지만, 실상은 기술의 논리에 따라 행동하고 있다. 
추천 알고리즘이 우리의 관심을 이끌고, 자동화된 시스템이 우리의 선택을 제한한다. 우리는 스스로 선택한다고 믿지만, 그 선택은 이미 누군가가 설계한 경로 위에 놓여 있다.
인공지능은 이러한 기술 지배의 정점에 서 있다. 단순한 계산 기계를 넘어, 인간의 사고방식을 흉내 내며 판단과 예측을 수행한다. 
의료에서는 진단을 돕고, 금융에서는 투자 전략을 제안하며, 예술에서는 새로운 창작물을 만들어 낸다. 그러나 이 모든 편리함 뒤에는 책임이라는 문제가 숨어 있다. 
인공지능이 잘못된 판단을 내렸을 때, 그 결과는 누구의 것인가? 인간이 만든 알고리즘이지만, 인간이 직접 개입하지 않은 결정이라면 책임의 주체는 흐려진다.
기술은 인간의 능력을 확장하지만 동시에 인간의 자율성을 약화시킨다. 인간은 점점 더 시스템의 일부로 편입되고, 그 시스템이 요구하는 속도와 효율에 맞춰 살아간다. 
느림, 비효율, 사색 같은 인간적인 요소는 점점 설 자리를 잃는다. 하지만 기술의 발전이 멈출 수 없다면, 인간이 할 일은 그 속도를 따라잡는 것이 아니라, 기술과의 관계를 다시 정의하는 것이다. 
기술은 목적이 아니라 도구이며, 인간의 가치를 보조해야 한다.
우리는 기술을 신으로 만들어선 안 된다. 기술은 인간의 손으로 만들어진 산물이고, 인간의 윤리 안에서 통제되어야 한다. 
인공지능이 아무리 발전해도, 인간의 감정과 도덕적 판단을 완벽히 대체할 수는 없다. 결국 중요한 것은 무엇을 할 수 있는가가 아니라 무엇을 해야 하는가이다. 
기술의 진보가 인간의 삶을 풍요롭게 만드는 방향으로 나아가려면, 우리는 다시 질문해야 한다. 이것이 인간에게 진정한 이익이 되는가?
"""

train_tokens = nltk.word_tokenize(train_text)

unigram = train_tokens
bigram = list(ngrams(train_tokens, 3))

unigram_freq = Counter(unigram)
bigram_freq = Counter(bigram)

In [74]:
generated_text = generate_text_trigram(seed_bigram=("인공지능이", "아무리"), trigram_freq=bigram_freq, max_len=200, max_sent=10)
generated_text

'인공지능이 아무리 발전해도, 인간의 감정과 도덕적 판단을 완벽히 대체할 수는 없다. 결국 중요한 것은 무엇을 할 수 있는가가 아니라 무엇을 해야 하는가이다. 기술의 진보가 인간의 삶을 풍요롭게 만드는 방향으로 나아가려면, 우리는 다시 질문해야 한다. 인공지능이 잘못된 판단을 내렸을 때, 그 결과는 누구의 것인가? 인간이 만든 알고리즘이지만, 인간이 언어를 이해하는 방식을 역으로 탐구하는 과정이기도 하다. 언어는 단순한 소통의 도구가 아니라 인간 사고의 본질이다. 언어를 분석한다는 것은 곧 인간의 생각을 해부하는 일이다. 그리고 인공지능은 지금 그 해부의 최전선에 서 있다. 단순한 계산 능력의 향상이 아니라, 인간이 할 일은 그 속도를 따라잡는 것이 아니라, 기술과의 관계를 다시 정의하는 것이다. 기술은 인간의 일상을 지배하고 있다. 인공지능은 이러한 기술 지배의 정점에 서 있다.'